[![image](https://binder.pangeo.io/badge_logo.svg)](https://gishub.org/leafmap-pangeo)

**Using time slider for visualizing timeseries images**

Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [ ]:
# !pip install leafmap

This notebook requires the ipyleaflet plotting backend. Folium is not supported.

In [ ]:
from leafmap import leafmap

First, you need to sign up a Planet account and get an API key. See https://developers.planet.com/quickstart/apis.
Uncomment the following line to pass in your API key.

In [ ]:
# os.environ["PLANET_API_KEY"] = "12345"

![](https://i.imgur.com/ipVJ4cb.gif)

Specify the map center and zoom level. 

In [ ]:
m = leafmap.Map(center=[38.2659, -103.2447], zoom=13)
m

Use the time slider to visualize Planet quarterly mosaic.

In [ ]:
m = leafmap.Map()
layers_dict = leafmap.planet_quarterly_tiles()
m.add_time_slider(layers_dict, time_interval=1)
m

Use the time slider to visualize basemaps.

In [ ]:
m = leafmap.Map()
m.clear_layers()
layers_dict = leafmap.basemap_xyz_tiles()
m.add_time_slider(layers_dict, time_interval=1)
m